In [1]:
# import libraries
import pandas as pd
import html
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen 
import urllib.request  
from tabula import read_pdf

In [2]:
my_page = requests.get(
    "https://www.railpassengers.org/all-aboard/tools-info/ridership-statistics/")
my_page.status_code

200

In [3]:
transit_soup = BeautifulSoup(my_page.content, 'html.parser')

In [14]:
station_list = []
url_val = []


q=0
for l in transit_soup.findAll('option'):
    station = l.text.strip()
    if q ==1:
        station_list.append(station)
        url_val.append(l['value'])
    if station == "Wyoming":
        q=1
    if station == "Yuma, AZ (YUM)":
        q=0



In [16]:
len(url_val)

545

In [26]:
df = pd.DataFrame({"Station":station_list,
                  "url":url_val})

df['url']="https://www.railpassengers.org/"+df['url']
df=df[1:]



In [32]:
city_list = []

for x in df['Station']:
    city_list.append(x[:x.find(",")])
    
df['city']=city_list 
df.head()

,Station,url,city
1,"Aberdeen, MD (ABE)",https://www.railpassengers.org//site/assets/fi...,Aberdeen
2,"Albany, OR (ALY)",https://www.railpassengers.org//site/assets/fi...,Albany
3,"Albany-Rensselaer, NY (ALB)",https://www.railpassengers.org//site/assets/fi...,Albany-Rensselaer
4,"Albion, MI (ALI)",https://www.railpassengers.org//site/assets/fi...,Albion
5,"Albuquerque, NM (ABQ)",https://www.railpassengers.org//site/assets/fi...,Albuquerque


In [36]:
#Download all pdfs

for i in range(0, len(df)):
#    print(states.iloc[i]['url'], states.iloc[i]['report type'])
    url=df.iloc[i]['url']
    r = requests.get(url)
    file_name="./station pdfs/"
    file_name=file_name+(df.iloc[i]['city'])+".pdf"
    try:
        with open(file_name, "wb") as code:
            code.write(r.content)
    except:
        print(df.iloc[i]['city'], "does not work")
    
    if i%50==0:
        print(df.iloc[i]['city']," done: ", str(len(df)-i), "remaining")

Aberdeen  done:  544 remaining
Boston - Route 128 / Westwood does not work
Bridgeport  done:  494 remaining
Croton-Harmon  done:  444 remaining
Fargo  done:  394 remaining
Hartford  done:  344 remaining
Lafayette  done:  294 remaining
Miami  done:  244 remaining
Minneapolis / St. Paul does not work
Old Town  done:  194 remaining
Reno  done:  144 remaining
Simi Valley  done:  94 remaining
Ventura  done:  44 remaining


In [40]:
## Read pdfs
## Needs updates for coordinates because not capturing all data for stations

station_table=pd.DataFrame()
years = ['2012','2013','2014','2015','2016','2017','2018']



for i in range(0, len(df)):
    city=df.iloc[i]['city']
    
    #need to define file directory
    file_name="./station pdfs/"+(df.iloc[i]['city'])+".pdf"
    #read pdf as new variable using tabula 
    try:
        df2=read_pdf(file_name, 
                        spreadsheet=True, 
                        area=(101.118,29.647,291.706,342.0)) 
                                ###need to widen coordinates###
        
    except FileNotFoundError:
        print(city, ": File Not Found")
        new_station=pd.DataFrame()
        new_station['Year'] = years
        new_station["city"]=city
        new_station["Ridership"]="N/A"
        df = pd.concat([df, new_station], sort=False)


    vals=[]
    #header becomes the 2012 value when reading in table
    try:
        x_list=df.columns.values.tolist()
        for j in x_list:
            x=j.replace(",","")

        x=float(x)
        vals.append(x)

        value_list = df2.values.tolist()
        for y in value_list:
            z=y[0]
            z=str(z)
            z=z.replace(",","")
            z=float(z)
            vals.append(z)

        new_station=pd.DataFrame()
        new_station['Year'] = years
        new_station['Ridership']=vals
        new_station["city"]=city
        df = pd.concat([df, new_station], sort=False)

    except AttributeError:
        #some stations have no ridership data for 2012-2018
        print("No Data")
        new_station=pd.DataFrame()
        new_station['Year'] = years
        new_station["city"]=city
        new_station["Ridership"]="N/A"
        df = pd.concat([df, new_station], sort=False)
        
    except ValueError:
        #some stations ended services during 2013-2018
        print("Partial Data")
        new_station=pd.DataFrame()
        new_station['Year'] = years
        new_station["city"]=city
        new_station["Ridership"]="Partial Data"
        df = pd.concat([df, new_station], sort=False)

Aberdeen
Partial Data
Albany
Partial Data
Albany-Rensselaer
Partial Data
Albion
Partial Data
Albuquerque
Partial Data
Alderson
Partial Data
Alexandria
Partial Data
Alliance
Partial Data
Alpine
Partial Data
Alton
Partial Data
Altoona
Partial Data
Amherst
Partial Data
Amsterdam
Partial Data
Anaheim
Partial Data
Ann Arbor
Partial Data
Anniston
Partial Data
Antioch-Pittsburg
Partial Data
Arcadia
Partial Data
Ardmore
Partial Data
Ardmore
Partial Data
Arkadelphia
Partial Data
Ashland
Partial Data
Ashland
Partial Data
Atlanta
Partial Data
Atmore
Partial Data
Auburn
Partial Data
Austin
Partial Data
Autumn Express [Selkirk]
Partial Data
Bakersfield
Partial Data
Baltimore - BWI Thurgood Marshall Airport
Partial Data
Baltimore - Penn Station
Partial Data
Bangor
Partial Data
Barstow
Partial Data
Battle Creek
Partial Data
Bay St. Louis
Partial Data
Beaumont
Partial Data
Bellingham
Partial Data
Bellows Falls
Partial Data
Benson
Partial Data
Berkeley
Partial Data
Berlin
Partial Data
Biloxi
Partial Da

Partial Data
North Philadelphia
Partial Data
Northampton
Partial Data
NY State Fair - Syracuse
Partial Data
Oakland - Coliseum Airport
Partial Data
Oakland - Jack London Sq
Partial Data
Oceanside
Partial Data
Okeechobee
Partial Data
Oklahoma City
Partial Data
Old Orchard Beach
Partial Data
Old Saybrook
Partial Data
Old Town
Partial Data
Omaha
Partial Data
Ontario
Partial Data
Orange
Partial Data
Oregon City
Partial Data
Oregon State Fair [Salem]
Partial Data
Orlando
Partial Data
Osceola
Partial Data
Ottumwa
Partial Data
Oxnard
Partial Data
Palatka
Partial Data
Paoli
Partial Data
Parkesburg
Partial Data
Pascagoula
Partial Data
Pasco
Partial Data
Paso Robles
Partial Data
Pauls Valley
Partial Data
Pensacola
Partial Data
Petersburg
Partial Data
Philadelphia - 30th Street Station
Partial Data
Picayune
Partial Data
Pittsburgh
Partial Data
Pittsfield
Partial Data
Plano
Partial Data
Plattsburgh
Partial Data
Pomona
Partial Data
Pontiac
Partial Data
Pontiac
Partial Data
Poplar Bluff
Partial Data

KeyboardInterrupt: 

In [ ]:
df.head()

In [22]:
station_table_final= station_table.merge(stations, on="city")

In [23]:
station_table_final.to_csv("Station Table.csv")